--- Day 4: Giant Squid ---
You're already almost 1.5km (almost a mile) below the surface of the ocean, already so deep that you can't see any sunlight. What you can see, however, is a giant squid that has attached itself to the outside of your submarine.

Maybe it wants to play bingo?

Bingo is played on a set of boards each consisting of a 5x5 grid of numbers. Numbers are chosen at random, and the chosen number is marked on all boards on which it appears. (Numbers may not appear on all boards.) If all numbers in any row or any column of a board are marked, that board wins. (Diagonals don't count.)

The submarine has a bingo subsystem to help passengers (currently, you and the giant squid) pass the time. It automatically generates a random order in which to draw numbers and a random set of boards (your puzzle input). For example:

7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1

22 13 17 11  0

 8  2 23  4 24

21  9 14 16  7

 6 10  3 18  5

 1 12 20 15 19


 3 15  0  2 22

 9 18 13 17  5

19  8  7 25 23

20 11 10 24  4

14 21 16 12  6


14 21 17 24  4

10 16 15  9 19

18  8 23 26 20

22 11 13  6  5

 2  0 12  3  7


After the first five numbers are drawn (7, 4, 9, 5, and 11), there are no winners, but the boards are marked as follows (shown here adjacent to each other to save space):


22 13 17 11  0         3 15  0  2 22        14 21 17 24  4

 8  2 23  4 24         9 18 13 17  5        10 16 15  9 19

21  9 14 16  7        19  8  7 25 23        18  8 23 26 20

 6 10  3 18  5        20 11 10 24  4        22 11 13  6  5

 1 12 20 15 19        14 21 16 12  6         2  0 12  3  7

After the next six numbers are drawn (17, 23, 2, 0, 14, and 21), there are still no winners:

22 13 17 11  0         3 15  0  2 22        14 21 17 24  4

 8  2 23  4 24         9 18 13 17  5        10 16 15  9 19

21  9 14 16  7        19  8  7 25 23        18  8 23 26 20

 6 10  3 18  5        20 11 10 24  4        22 11 13  6  5

 1 12 20 15 19        14 21 16 12  6         2  0 12  3  7

Finally, 24 is drawn:

22 13 17 11  0         3 15  0  2 22        14 21 17 24  4

 8  2 23  4 24         9 18 13 17  5        10 16 15  9 19

21  9 14 16  7        19  8  7 25 23        18  8 23 26 20

 6 10  3 18  5        20 11 10 24  4        22 11 13  6  5

 1 12 20 15 19        14 21 16 12  6         2  0 12  3  7

At this point, the third board wins because it has at least one complete row or column of marked numbers (in this case, the entire top row is marked: 14 21 17 24 4).

The score of the winning board can now be calculated. Start by finding the sum of all unmarked numbers on that board; in this case, the sum is 188. Then, multiply that sum by the number that was just called when the board won, 24, to get the final score, 188 * 24 = 4512.

To guarantee victory against the giant squid, figure out which board will win first. What will your final score be if you choose that board?

current issue: arr is a list of numbers, but it needs to be a list of lists of numbers

In [62]:
import numpy as np

with open('day4_puzzle_input.txt') as fp:
    string = fp.readline().strip()
    tmp_string = ''
    winning_numbers = []    # list to hold winning values in drawn order
    for i in range(len(string)):
        if ord(string[i]) != 44:
            tmp_string = tmp_string + string[i]
        else:
            winning_numbers.append(int(tmp_string))
            tmp_string = ''
    string = fp.readline().strip()
    arr = []
    if string == '':
        string = fp.readline().strip()
    while string:
        tmp_string = ''
        tmp_list = []
        for i in range(len(string)):
            if ord(string[i]) != 32:
                tmp_string = tmp_string + string[i]
            else:
                if tmp_string != '':
                    tmp_list.append(int(tmp_string))
                    tmp_string = ''
        else:
            tmp_list.append(int(tmp_string))
        arr.append(tmp_list)
        string = fp.readline().strip()
        if string == '':
            string = fp.readline().strip()

bingo_array = np.array(arr)
fastest_board = []  # a list to hold the integer value of how many turns each board takes
ones_array = np.ones(5,dtype='int32')
score = []  # The score of each bingo board

for i in range(int(len(bingo_array)/5)):
    turns = 0   # integer to track how many bingo calls must be made for an individual board to win
    turns2 = 0
    bingo_board_horizontal = np.copy(bingo_array[i*5:i*5+5])
    bingo_board_vertical = np.rot90(bingo_board_horizontal,k=1,axes=(0,1))
    bingo_board = np.concatenate((bingo_board_horizontal,bingo_board_vertical))
    for j in range(len(winning_numbers)):
        break_point = 0
        if winning_numbers[j] in bingo_board:
            turns +=1
            bingo_board[bingo_board - winning_numbers[j] == 0] = -1
            for k in range(len(bingo_board)):
                comparison = np.multiply(bingo_board[k],-1) == ones_array   # checks if a bingo has been achieved
                equal_arrays = comparison.all()
                if equal_arrays:    # If any board has a bingo, saves how many turns it took
                    fastest_board.append(turns+turns2)
                    break_point = 1
                    bingo_board[bingo_board == -1] = 0
                    score.append(np.sum(bingo_board[:5]) * winning_numbers[j])
                    break
        else:
            turns2 +=1

        if break_point == 1:
            break

d = {}  # dictionary to store boards by the amount of turns it takes to win
for i in range(len(score)):
    d[fastest_board[i]] = score[i]

for i in range(len(fastest_board)-1):   # organize the boards to find the one that finishes in the quickest time
    if fastest_board[i] < fastest_board[i+1]:
        tmp = fastest_board[i] 
        fastest_board[i] = fastest_board[i+1]
        fastest_board[i+1] = tmp

print(f"The fastest board will finish in {fastest_board[-1]} turns with a score of {d[fastest_board[-1]]}")

The fastest board will finish in 29 turns with a score of 25410


--- Part Two ---
On the other hand, it might be wise to try a different strategy: let the giant squid win.

You aren't sure how many bingo boards a giant squid could play at once, so rather than waste time counting its arms, the safe thing to do is to figure out which board will win last and choose that one. That way, no matter which boards it picks, it will win for sure.

In the above example, the second board is the last to win, which happens after 13 is eventually called and its middle column is completely marked. If you were to keep playing until this point, the second board would have a sum of unmarked numbers equal to 148 for a final score of 148 * 13 = 1924.

Figure out which board will win last. Once it wins, what would its final score be?

In [63]:
for i in range(len(fastest_board)-1):
    if fastest_board[i] > fastest_board[i+1]:
        tmp = fastest_board[i]
        fastest_board[i] = fastest_board[i+1]
        fastest_board[i+1] = tmp

print(f"The slowest board will finish in {fastest_board[-1]} turns with a score of {d[fastest_board[-1]]}")

The slowest board will finish in 83 turns with a score of 2730
